Neural Network to classify the MNIST Hand written digit.

In [1]:
# Importing the library
import tensorflow as tf
from tensorflow import keras

import IPython
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [2]:
# Load the data from dataset 
(img_train, label_train), (img_test, label_test) = keras.datasets.mnist.load_data()

In [3]:
# Finding the shape of 
img_train[0].shape

(28, 28)

In [4]:
#Normalize pixel values between 0 and 1
img_train=img_train/255.0
img_test=img_test/255.0

In [5]:
#reshaping the data
img_train=img_train.reshape(len(img_train),28,28,1)
img_test=img_test.reshape(len(img_test),28,28,1)

In [6]:
#
def build_model(hp):  
  model = keras.Sequential([
    # Tune the number of units in the first CNN layer
    # Choose an optimal value between 32-128
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),

    # Tune the number of units in the second CNN layer
    # Choose an optimal value between 32-12648
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    # Tune the number of units in the first Dense layer
    # Choose an optimal value between 32-128
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    #last dense layer
    keras.layers.Dense(10, activation='softmax')
  ])
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-1,1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [7]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=10,directory='output',project_name="Mnist")

INFO:tensorflow:Reloading Oracle from existing project output/Mnist/oracle.json
INFO:tensorflow:Reloading Tuner from output/Mnist/tuner0.json


In [8]:
tuner_search.search(img_train,label_train,epochs=3,validation_split=0.1)

INFO:tensorflow:Oracle triggered exit


In [9]:
# Get the optimal hyperparameters
model=tuner_search.get_best_models(num_models=1)[0]

In [10]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 64)        640       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 22, 22, 48)        76848     
_________________________________________________________________
flatten (Flatten)            (None, 23232)             0         
_________________________________________________________________
dense (Dense)                (None, 64)                1486912   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                650       
Total params: 1,565,050
Trainable params: 1,565,050
Non-trainable params: 0
_________________________________________________________________


In [12]:
model.fit(img_train, label_train, epochs=30, validation_split=0.1, initial_epoch=3)

Epoch 4/30
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0037 - accuracy: 0.9988 - val_loss: 0.0901 - val_accuracy: 0.9892
Epoch 5/30
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0041 - accuracy: 0.9986 - val_loss: 0.0901 - val_accuracy: 0.9890
Epoch 6/30
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0031 - accuracy: 0.9989 - val_loss: 0.0920 - val_accuracy: 0.9898
Epoch 7/30
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0036 - accuracy: 0.9988 - val_loss: 0.0855 - val_accuracy: 0.9903
Epoch 8/30
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0034 - accuracy: 0.9990 - val_loss: 0.0794 - val_accuracy: 0.9905
Epoch 9/30
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0011 - accuracy: 0.9996 - val_loss: 0.0939 - val_accuracy: 0.9895
Epoch 10/30
1688/1688 [==============================] - 7s 4ms/step - loss: 0.0044 - accuracy: 0.9987 - val_loss: 0.0915 - val_accuracy